# BigQuery + Cloud Functions: how to run your queries as soon as a new Google Analytics table is available

https://towardsdatascience.com/bigquery-cloud-functions-how-to-run-your-queries-as-soon-as-a-new-google-analytics-table-is-17fbb62f8aaa



In [ ]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

In [ ]:
project_number=!gcloud projects describe $(gcloud config get-value project) --format='value(projectNumber)'
PROJECT_NUMBER = project_number[0]
PROJECT_NUMBER

In [3]:
DATASET_NAME = "bq_eventarc_queries_demo"
TABLE_NAME = "loan_201"

In [ ]:
DEFAULT_SVC_ACCOUNT_EMAIL=f"{PROJECT_NUMBER}-compute@developer.gserviceaccount.com"
DEFAULT_SVC_ACCOUNT_EMAIL

## 0 - setup

1. Setup GCP - run `00_setup_env.sh`  - enable APIs, create GCS bucket 
3. Setup BQ - run `01_setup_bq.sh` - ingest sample data to GCS bucket, create target BQ dat

### Create Service account


refs 

* https://cloud.google.com/bigquery/docs/access-control
* 

In [5]:
# TODO @justinjm - add creation and setup of service account 

# # TODO - Set the name of your project
# PROJECT_ID="your-project-id" 
# # TODO - Set the name of your service account
# SA_NAME="bq-scheduler" 
# SA_EMAIL="${SA_NAME}@${PROJECT_ID}.iam.gserviceaccount.com" 

# # TODO - set target user that will schedule BQ queries
# USER_EMAIL="email@company.com" # can also be group 

# # Create the service account
# gcloud iam service-accounts create $SA_NAME --project $PROJECT_ID

# ## service account access --------------------------------------------
# ## Grant the service account project editor permissions
# ## or `roles/bigquery.jobUser` if minimal required
# gcloud projects add-iam-policy-binding $PROJECT_ID \
#   --member "serviceAccount:${SA_EMAIL}" \
#   --role "roles/bigquery.admin" \
#   --condition="None"

# ## user group access --------------------------------------------
# gcloud projects add-iam-policy-binding $PROJECT_ID \
#   --member="user:${USER_EMAIL}" \
#   --role="roles/bigquery.user"  \
#   --condition="None"

# gcloud projects add-iam-policy-binding $PROJECT_ID \
#   --member="user:${USER_EMAIL}" \
#   --role="roles/iam.serviceAccountViewer" \
#   --condition="None"

# ## give users/groups aaccess 
# ## https://cloud.google.com/iam/docs/service-account-permissions
# gcloud iam service-accounts add-iam-policy-binding "${SA_NAME}@${PROJECT_ID}.iam.gserviceaccount.com" \
#   --member="user:${USER_EMAIL}" \
#   --role="roles/iam.serviceAccountUser" \
#   --condition="None"


## Create BQ  scheduled query 

#TODO @justinjm - create scheduled query as part of setup BQ
# https://cloud.google.com/bigquery/docs/scheduling-queries#python_1


refs 

* https://cloud.google.com/bigquery/docs/scheduling-queries#python_1
* https://cloud.google.com/bigquery/docs/access-control
* https://cloud.google.com/iam/docs/manage-access-service-accounts#iam-view-access-sa-gcloud

API

* https://cloud.google.com/python/docs/reference/bigquerydatatransfer/latest/google.cloud.bigquery_datatransfer_v1.types.TransferConfig

In [ ]:
from google.cloud import bigquery_datatransfer

transfer_client = bigquery_datatransfer.DataTransferServiceClient()

# The project where the query job runs is the same as the project
# containing the destination dataset.
project_id = PROJECT_ID
dataset_id = DATASET_NAME

# This service account will be used to execute the scheduled queries. Omit
# this request parameter to run the query as the user with the credentials
# associated with this client.
service_account_name = DEFAULT_SVC_ACCOUNT_EMAIL

# Use standard SQL syntax for the query.
query_string = f"""
SELECT * FROM `{PROJECT_ID}.{DATASET_NAME}.{TABLE_NAME}` LIMIT 10
"""

parent = transfer_client.common_project_path(project_id)

transfer_config = bigquery_datatransfer.TransferConfig(
    destination_dataset_id=dataset_id,
    display_name="BigQuery EventArc Driven Query",
    data_source_id="scheduled_query",
    params={
        "query": query_string,
        "destination_table_name_template": "processed_{run_date}",
        "write_disposition": "WRITE_TRUNCATE",

    },
    # schedule="on-demand",
)

transfer_config = transfer_client.create_transfer_config(
    bigquery_datatransfer.CreateTransferConfigRequest(
        parent=parent,
        transfer_config=transfer_config,
        service_account_name=service_account_name,
    )
)

print("Created scheduled query '{}'".format(transfer_config.name))

## 1 - cloud logging filter 


Demo Version:

```txt
protoPayload.serviceData.jobCompletedEvent.job.jobConfiguration.load.destinationTable.datasetId="bq_eventarc_queries_demo"
protoPayload.serviceData.jobCompletedEvent.job.jobConfiguration.load.destinationTable.projectId="demos-vertex-ai"
protoPayload.methodName="jobservice.jobcompleted"
protoPayload.serviceData.jobCompletedEvent.job.jobConfiguration.load.destinationTable.tableId:"loan_201"
```


Google Analytics Version: 

```txt
protoPayload.serviceData.jobCompletedEvent.job.jobConfiguration.load.destinationTable.datasetId="[REPLACE_WITH_YOUR_DATASET_ID]"
protoPayload.serviceData.jobCompletedEvent.job.jobConfiguration.load.destinationTable.projectId="REPLACE_WITH_YOUR_PROJECT_ID"
protoPayload.authenticationInfo.principalEmail="analytics-processing-dev@system.gserviceaccount.com"
protoPayload.methodName="jobservice.jobcompleted"
protoPayload.serviceData.jobCompletedEvent.job.jobConfiguration.load.destinationTable.tableId:"ga_sessions"
NOT protoPayload.serviceData.jobCompletedEvent.job.jobConfiguration.load.destinationTable.tableId:"ga_sessions_intraday"
```

refs

* https://cloud.google.com/logging/docs/view/building-queries
* https://cloud.google.com/logging/docs/view/logging-query-language

In [2]:
!PROJECT_ID=$(gcloud config get-value project)
!PROJECT_NUMBER=$(gcloud projects describe $(gcloud config get-value project) --format='value(projectNumber)')

In [ ]:
## Create Pub/Sub topic
!gcloud pubsub topics create bq-load-events-topic

In [ ]:
## create log sink filter based on query above 
!gcloud logging sinks create bq-load-events-sink "pubsub.googleapis.com/projects/${PROJECT_ID}/topics/bq-load-events-topic" \
    --log-filter='protoPayload.serviceData.jobCompletedEvent.job.jobConfiguration.load.destinationTable.datasetId=\"bq_eventarc_queries_demo\" AND protoPayload.serviceData.jobCompletedEvent.job.jobConfiguration.load.destinationTable.projectId=\"demos-vertex-ai\" AND protoPayload.methodName=\"jobservice.jobcompleted\" AND protoPayload.serviceData.jobCompletedEvent.job.jobConfiguration.load.destinationTable.tableId:\"loan_201\"'

In [ ]:
# grant `serviceAccount:service-PROJECT_NUMBER@gcp-sa-logging.iam.gserviceaccount.com` the Pub/Sub Publisher role on the topic.
# More information about sinks can be found at https://cloud.google.com/logging/docs/export/configure_export
!gcloud projects add-iam-policy-binding $PROJECT_ID \
  --member="serviceAccount:service-${PROJECT_NUMBER}@gcp-sa-logging.iam.gserviceaccount.com" \
  --role="roles/pubsub.publisher" \
  --project=$PROJECT_ID \
  --condition=None

## 3 - Create Cloud Function 



#### Create Cloud Function

Create and deploy a Cloud function from the source code in the [functions](functions/) directory:



### Create CF files 

First we create necessary files 

In [4]:
# ! rm -rf functions/
# !mkdir functions

In [ ]:
# %%writefile functions/main.py 
# import time
# from google.protobuf.timestamp_pb2 import Timestamp
# from google.cloud import bigquery_datatransfer_v1

# def runQuery (parent, requested_run_time):
#     client = bigquery_datatransfer_v1.DataTransferServiceClient()
#     projectid = 'XXXXXXXXXX' # Enter your projectID here
#     transferid = 'XXXXXXXXXX'  # Enter your transferId here
#     parent = client.project_transfer_config_path(projectid, transferid)
#     start_time = bigquery_datatransfer_v1.types.Timestamp(seconds=int(time.time() + 10))
#     response = client.start_manual_transfer_runs(parent, requested_run_time=start_time)
#     print(response)
    
# # do not forget to put google-cloud-bigquery-datatransfer==1 in the requirements.txt

In [ ]:
# %%writefile functions/requirements.txt
# google-cloud-bigquery-datatransfer==1

In [ ]:
# !gcloud functions deploy bq-eventarc-driven-queries-demo \
#   --gen2 \
#   --region=us-central1 \
#   --runtime=python311 \
#   --source=functions/ \
#   --entry-point=run \
#   --trigger-topic=YOUR_TOPIC_NAME \
#   --timeout=3600 \
#   --no-allow-unauthenticated